In [1]:
import jieba
import numpy as np
import pandas as pd

In [2]:
pos = pd.read_csv('../Desktop/weather_pos.txt', encoding='UTF-8', header=None)

In [3]:
neg = pd.read_csv('../Desktop/weather_neg.txt', encoding='UTF-8', header=None)

In [4]:
pos['words'] = pos[0].apply(lambda x: jieba.lcut(x))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Oscar\AppData\Local\Temp\jieba.cache
Loading model cost 0.677 seconds.
Prefix dict has been built successfully.


In [5]:
neg['words'] = neg[0].apply(lambda x: jieba.lcut(x))

In [6]:
neg

,0,words
0,没心没肺一点就是爱自己,"[没心没肺, 一点, 就是, 爱, 自己]"
1,你是不想活了,"[你, 是, 不想, 活, 了]"
2,有个差不多的人来个差不多的触电,"[有个, 差不多, 的, 人来, 个, 差不多, 的, 触电]"
3,那你刚才说你高一,"[那, 你, 刚才, 说, 你, 高一]"
4,博彩通百度最新消息博彩到JQK365娱乐城,"[博彩, 通, 百度, 最新消息, 博彩, 到, JQK365, 娱乐城]"
...,...,...
22508,设定闹钟铃声,"[设定, 闹钟, 铃声]"
22509,把空调的风调小,"[把, 空调, 的, 风调, 小]"
22510,我让你变得温柔一点说话不要那么过分,"[我, 让, 你, 变得, 温柔, 一点, 说话, 不要, 那么, 过分]"
22511,小型跳圆圈舞,"[小型, 跳, 圆圈, 舞]"


In [7]:
x = np.concatenate((pos['words'], neg['words']))

In [8]:
y = np.concatenate((np.ones(len(pos)),np.zeros(len(neg))))

In [9]:
from gensim.models.word2vec import Word2Vec

In [10]:
word2vec = Word2Vec(x, size=300, window=3, min_count=5, sg=1, hs=1, iter=10, workers=25)

In [11]:
word2vec.save('../Desktop/word2vec.model')

In [12]:
word2vec.wv['温度']

array([-0.44340527, -0.46484488,  0.2581572 ,  0.21782063,  0.00132482,
        0.20777151,  0.65886456,  0.33752787, -0.3786229 ,  0.20326719,
        0.42066148, -0.13763933,  0.51602566,  0.18755844, -0.07084246,
        0.24245203,  0.23999532,  0.04527772,  0.0873777 , -0.03502242,
        0.4726013 ,  0.48828664,  0.07702077,  0.03528466, -0.3649241 ,
        0.18934588,  0.46830156,  0.03844164,  0.76555914,  0.03859922,
       -0.09633848, -0.12663956,  0.03962589, -0.24343632,  0.03257497,
        0.21635148,  0.07382055, -0.00381618,  0.1388726 , -0.23641074,
        0.21457686,  0.1909143 , -0.36611608,  0.39738092, -0.33045793,
       -0.00537591,  0.35812   ,  0.11014543,  0.05910615, -0.05027646,
        0.28114513, -0.19536595, -0.18062305, -0.06461969, -0.22762798,
       -0.06819354, -0.0015293 , -0.16626519, -0.24745505,  0.01657421,
       -0.12141601,  0.15320075,  0.20736527,  0.28431153,  0.00806244,
        0.09121972,  0.11823659, -0.63201326, -0.09189388, -0.22

In [13]:
def total_vector(words):
    vec = np.zeros(300).reshape((1, 300))
    for word in words:
        try:
            vec += word2vec.wv[word].reshape((1, 300))
        except KeyError:
            continue
    return vec

In [14]:
train_vec = np.concatenate([total_vector(words) for words in x])

In [15]:
train_vec

array([[-2.15116006,  0.2649736 , -0.53555788, ...,  0.27691792,
         0.78636618,  0.489801  ],
       [-0.86660157, -0.06048669, -0.04459131, ...,  0.33700325,
         0.51765865,  0.34426135],
       [-0.84104186, -0.45222624, -0.06007541, ...,  0.39277741,
         0.24993864,  0.16592727],
       ...,
       [-0.83536033,  1.00087314, -0.13343931, ..., -0.14823668,
         0.61255995, -1.65132197],
       [-0.05177539,  0.34515835,  0.28901224, ...,  0.04726395,
         0.27061148, -0.26866695],
       [ 0.16190385,  0.28474975, -0.12673797, ..., -0.15524003,
         0.22284457, -1.08251442]])

In [16]:
from sklearn.externals import joblib
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC

c:\users\oscar\appdata\local\programs\python\python38\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [17]:
model = SVC(kernel='rbf', verbose=True)

In [18]:
model.fit(train_vec, y)

[LibSVM]

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=True)

In [19]:
joblib.dump(model, '../Desktop/weather_svm.pkl')

['../Desktop/weather_svm.pkl']

In [20]:
model = joblib.load('../Desktop/weather_svm.pkl')

In [21]:
def svm_predict(query):
    words = jieba.lcut(str(query))
    words_vec = total_vector(words)
    result = model.predict(words_vec)
    if int(result) == 1:
        print('类别：天气')
    elif int(result) == 0:
        print('类别：其他')

In [22]:
svm_predict('明天冷不冷')

类别：天气


In [23]:
svm_predict('我爱你妈卖麻花情')

类别：其他


In [24]:
svm_predict('明天有雨吗')

类别：天气


In [25]:
svm_predict('现在几点钟了')

类别：其他
